## Classifying Clapper Rails and King Rails Based on their Kek Calls

This project will analyze acoustic markers derived from audio recordings of Clapper Rails and King Rails in an attempt to classify the species.  
Sample sounds from these call can be listened to at The Cornell Lab:
 - Clapper Rail Sounds: https://www.allaboutbirds.org/guide/Clapper_Rail/sounds
 - King Rail Sounds: https://www.allaboutbirds.org/guide/King_Rail/sounds

Original work on this dataset is documented at the Wiley Online Library: https://onlinelibrary.wiley.com/doi/10.1002/ece3.4711  
A copy of this research is also included in this git repository as "Ecology and Evolution - 2018 - Stiffler.pdf"

During this analysis we will review the following questions:
 - Can we reproduce the basic findings regarding optimal machine learning models for this classification?
 - Are there improvements that can be made in the outcomes?

https://github.com/carl-schick-ds/rails.git

***
### Setup
Import needed libraries.  Unless otherwise noted, all libraries are available in the baseline conda environment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import CategoricalDtype
from IPython.display import display

***
### Data Collection
Data collection was not required for this project since the original dataset is available online in CSV format and is included in this repository as "KIRACLRA.csv".  
I reached out to the original researches in an attempt to obtain the source audio recordings that were used to produce the audio markers in the dataset, but those recordings are no longer available.  

It's feasible that improvements could be made to the classification model by using different audio markers, but this is beyond the scope of this analysis and, in any regard, is not possible due to the unavailability of the source audio recordings.

### Data Loading and Cleaning

#### Load CSVs
Load the data from the csv files and run a quick review of the data for validity checks

In [ ]:
rails_df = pd.read_csv('KIRACLRA.csv')
rails_df.head()

In [ ]:
rails_df.info()

***
Generate descriptive statistics for both the Clapper Rail and King Rail recordings.  
These were compared back to the original research to confirm we are starting from the same place.
***

In [ ]:
print("Clapper Rails Descriptive Statistics")
display(rails_df[rails_df['Species'] == 'CLRA'].describe().round(decimals=0).astype(int))
print("King Rails Descriptive Statistics")
display(rails_df[rails_df['Species'] == 'KIRA'].describe().round(decimals=0).astype(int))

#### Clean the Data

Confirm the target column, Species, review the counts, rename the values, and reset the column as a categorical type.

In [ ]:
## Define our Categorical Dtype in order to ensure the Clapper Species is the positive value during classification
species_cat = CategoricalDtype(categories=['King', 'Clapper'], ordered=True)  

print(rails_df['Species'].value_counts())
rails_df['Species'] = rails_df['Species'].apply(lambda x: 'Clapper' if x == 'CLRA' else 'King')
rails_df['Species'] = rails_df['Species'].astype(species_cat)
print(rails_df['Species'].value_counts())
# print()
# print(rails_df['Species'].cat.codes)
print()
display(rails_df.info())

***
Review the correlation of the parameters to determine if any parameters can be dropped in order to simply the models.  
Generate both a correlation table and a heatmap.
***

In [ ]:
corr = rails_df.corr()
display(corr)

In [ ]:
# Set the self-correlation cells (the diaganols) to zero to minimize their prominence during visual review
np.fill_diagonal(corr.values, 0)

# Set the palette to 'coolwarm' and the min/max to the extreme ends of valid correlation values
colormap = sns.color_palette("coolwarm")
vmin, vmax = (-1, 1)

# Display the heatmap
sns.heatmap(corr, center=(vmin + vmax) / 2, vmin=vmin, vmax=vmax, cmap=colormap)
'';

Both the correlation table and the heatmap indicate a strong correlation between the Bandwidth (BW) markers and one of the Frequency ranges.  
Specifically, `IGR BW (Hz)` is highly correlated with `Q3 Freq Hz` (*r* $\approx$ .75) and `BW 90% (Hz)` is highly correlated with `Freq 95% (Hz)` (*r* $\approx$ .89)  

<mark>Is this next statement true?  I ran most of the models with and without the Bandwidth columns and the results were similar or worse when they were included.  
Are there considerations other than the *r* value that I should be reviewing for the claims below?</mark>  

Parameters with correlations above .70 - .75 will have a negligible impact on the model results since their values are appropriately accounted for in another parameter.  At times, their inclusion could actually produce worse outcomes since they can add unnecessary noise to the model generation.  For this reason, we will drop both of the Bandwidth (BW) markers from the dataset.  

***
Drop the Bandwidth (BW) columns due to their high correlation with other parameters.
***

In [ ]:
rails_df.drop(['IQR BW (Hz)', 'BW 90% (Hz)'], axis=1, inplace=True)
display(rails_df.head())

### Data Exploration

#### Visualize Parameter Realtionships with Species Classification
The relationship of each parameter with the Species classification is visualized in a variety of formats: historgrams, boxplots, violinplots, and scatterplots.

In [ ]:
feature_columns = rails_df.drop('Species', axis=1).columns.to_list()

fig, axes = plt.subplots(5,5, figsize=(25,25))
sns.set_style('whitegrid')

for i in range(len(feature_columns)):
    sns.histplot(data=rails_df, x=feature_columns[i], bins = 30, hue='Species', ax=axes[0,i])
    sns.kdeplot(data=rails_df, x=feature_columns[i], hue='Species', ax=axes[1,i])
    sns.boxplot(data=rails_df, x=feature_columns[i], y='Species', ax=axes[2,i])
    sns.violinplot(data=rails_df, x=feature_columns[i], y='Species', hue='Species', legend=False, ax=axes[3,i])
    sns.scatterplot(data=rails_df, x=feature_columns[i], y='Species', hue='Species', legend=False, ax=axes[4,i])

for i in range(len(axes[3])):
    axes[3,i].legend().remove()
'';

<mark>
I could use some help interpreting the above visualizations.  At a high level, it looks like there is a lot of overlap between the data values and the Species class, which would make modeling difficult.  The kdeplots show some bright spots, where some parameters (Freq 5% and Freq 95% in particular) clearly have values with higher occurrences in one species vs. another.  
<br>
<br>
But in general, how should I be interpreting this?  Does it give any indication of which model type, or which model in particular, would be best?  I'm not sure what the clear takeaways are, or whether I should be exploring anything differently in order to guide model selection.  A best-guess takeaway is below.
</mark>

As shown in the charts above, there is quite a bit of overlap in the Clapper and King data points.  Since parametric models such as Linear and Quadratic Discriminant Analysis and Logistic Regression depend on the data to follow a limited (fixed) parameter function, it's probable that those models will have a difficult time with the classification.  However, non-parametric models such K-Nearest Neighbors and Support Vector Machines that make no assumptions on the functional form of the data may be able to find a wide range of patterns to help aid a successful classification.  Let's get to it!

### Model Preparation

While not a huge variation in the range of the numeric values for the parameters, it' still a good idea to normalize the data.  
We will use a standard scaler for normalization, where `new_value = (orig_value - mean) / std-dev`.  

**Note: This scaling was not done during the original study and may help with yielding improved results.**

In addition, let's extract the features and target into generic X and y variables.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(rails_df.drop('Species', axis=1)), columns=rails_df.drop('Species', axis=1).columns)
# X = rails_df.drop('Species', axis=1)
y = rails_df['Species']

***
Split the data into test and train datasets, with 30% reserved for testing.
***

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=77)

***
Import the methods we will use for model evaluation
***

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score, roc_curve, auc
from sklearn.metrics import make_scorer, precision_score, recall_score, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay
plt.style.use('default')
pd.options.display.float_format = '{:,.3f}'.format

### Scoring Overview
We will use the same scoring metrics used in the original study.  Specifically, we will track...
 - **Accuracy**: the ratio of correctly predicted observations to the total observations.  `(TP + TN) / (TP + FP + TN + FN)`
 - **Precision**: the ratio of correctly predicted positive observations to the total predicted positive observations.  `TP / (TP + FP)`
 - **Sensitivity**: the ratio of correctly predicted positive observations to the total actual positive observations.  `TP / (TP + FN)`
 - **Specificity**: the ratio of correctly predicted negative observations to the total actual negative observations.  `TN / (TN + FP)`
 - **Area Under the Curve (AUC)**: estimated area under the `ROC curve`.  Used as a general measure of aggregated classification performance. 
 - **Cohen's kappa (*K*)**: similar to Accuracy, but takes into account the possibility of the classification happening by chance.

Reference Notes:  
 - `TP`: True Positive (correctly classified a Clapper rail as a Clapper rail)
 - `TN`: True Negative (correctly classified a King rail as a King rail)
 - `FP`: False Positive, Type I error (incorrectly classified a King rail as a Clapper rail)
 - `FN`: False Negative, Type II error (incorrectly classified a Clapper rail as a King rail)
 - `TPR`: True Positive Rate (the ratio of True Positives (`TP`) to the total actual positive observations; equal to Sensitivity)
 - `FPR`: False Positive Rate (the ratio of False Positives (`FP`) to the total actual negative observations; equal to 1 - Specificity)
 - `ROC curve`: graphical plot of the True Positive Rate (`TPR`) against the False Positive Rate (`FPR`) at various threshold settings (ranging 0.0 - 1.0)
<br><br>
 - Sensitivity is equal to the Recall of the positive class (Clapper)  
 - Specificity is equal to the Recall of the negative class (King)  

The outcomes from the original study are shown in the image below.

![](orig_outcomes.jpg)

### Model Selection

#### Parametric Models

***
Linear Discriminant Analysis
***

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

## LDR
ldr = LinearDiscriminantAnalysis()
ldr.fit(X_train,y_train)

# Training summary
print("Training Score:", round(ldr.score(X_train, y_train),3))
print("Coefficients:", ldr.coef_)
print("Intercept:", ldr.intercept_)

In [ ]:
# Make pedictions
predictions = ldr.predict(X_test)

# Get predicted probabilities
# Although Clapper is the positive class, it's the first class returned by predict_proba
y_pred_proba = ldr.predict_proba(X_test)
y_pred_proba_Clapper = ldr.predict_proba(X_test)[::,0]
y_pred_proba_King = ldr.predict_proba(X_test)[::,1]

# Calculate scores
class_rpt = classification_report(y_test, predictions, digits=3, output_dict=True)
fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba_Clapper, pos_label='Clapper')
scr_accuracy = round(class_rpt['accuracy'],3)
scr_precision = round(class_rpt['Clapper']['precision'],3)
scr_sensitivity = round(class_rpt['Clapper']['recall'], 3)
scr_specificity = round(class_rpt['King']['recall'], 3)
scr_auc = round(auc(fpr, tpr), 3)
scr_k = round(cohen_kappa_score(y_test, predictions), 3)

print("Alt AUC:", scr_auc_alt)

results_cols = ['Class', 'Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'AUC', 'K']
ldr_results = pd.DataFrame(columns=results_cols)
ldr_results.loc['Linear DFA'] = ['N', scr_accuracy, scr_precision, scr_sensitivity, scr_specificity, scr_auc, scr_k]
display(ldr_results)

# Display charts
fig, axes = plt.subplots(2 ,figsize=(5,8))
fig.tight_layout(h_pad=5, w_pad=5)

cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix= cm, display_labels=ldr.classes_)
disp.plot(ax=axes[0])
axes[1].plot(fpr,tpr,label="LDR AUC="+'{:.3f}'.format(scr_auc))
axes[1].plot([0, 1], [0, 1],linestyle='dashed', color='gray', label="Random classifier")
axes[1].set_xlabel('False Positive Rate (FPR)')
axes[1].set_ylabel('True Positive Rate (TPR)')
plt.legend(loc=4)
'';

***
Quadratic Discriminant Analysis
***

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

## QDR
qdr = QuadraticDiscriminantAnalysis(store_covariance=True)
qdr.fit(X_train,y_train)

# Training summary
print("Training Score:", round(qdr.score(X_train, y_train),3))


In [ ]:
# Make predictions
predictions = qdr.predict(X_test)

# Get predicted probabilities
# Although Clapper is the positive class, it's the first class returned by predict_proba
y_pred_proba = qdr.predict_proba(X_test)
y_pred_proba_Clapper = qdr.predict_proba(X_test)[::,0]
y_pred_proba_King = qdr.predict_proba(X_test)[::,1]

# Calculate scores
class_rpt = classification_report(y_test, predictions, digits=3, output_dict=True)
fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba_Clapper, pos_label='Clapper')
scr_accuracy = round(class_rpt['accuracy'],3)
scr_precision = round(class_rpt['Clapper']['precision'],3)
scr_sensitivity = round(class_rpt['Clapper']['recall'], 3)
scr_specificity = round(class_rpt['King']['recall'], 3)
scr_auc = round(auc(fpr, tpr), 3)
scr_k = round(cohen_kappa_score(y_test, predictions), 3)

results_cols = ['Class', 'Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'AUC', 'K']
qdr_results = pd.DataFrame(columns=results_cols)
qdr_results.loc['Quadratic DFA'] = ['N', scr_accuracy, scr_precision, scr_sensitivity, scr_specificity, scr_auc, scr_k]
display(qdr_results)

# Make charts
fig, axes = plt.subplots(2 ,figsize=(5,8))
fig.tight_layout(h_pad=5, w_pad=5)

cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix= cm, display_labels=qdr.classes_)
disp.plot(ax=axes[0])
axes[1].plot(fpr,tpr,label="QDR AUC="+'{:.3f}'.format(scr_auc))
axes[1].plot([0, 1], [0, 1],linestyle='dashed', color='gray', label="Random classifier")
axes[1].set_xlabel('False Positive Rate (FPR)')
axes[1].set_ylabel('True Positive Rate (TPR)')
plt.legend(loc=4)
'';

***
Logistic Regression
***

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

# Training summary
print("Training Score:", round(lr.score(X_train, y_train),3))
print("Coefficients:", lr.coef_)
print("Intercept:", lr.intercept_)

In [ ]:
# Make predictions
predictions = lr.predict(X_test)

# Get predicted probabilities
# Although Clapper is the positive class, it's the first class returned by predict_proba
y_pred_proba = lr.predict_proba(X_test)
y_pred_proba_Clapper = lr.predict_proba(X_test)[::,0]
y_pred_proba_King = lr.predict_proba(X_test)[::,1]

# Calculate scores
class_rpt = classification_report(y_test, predictions, digits=3, output_dict=True)
fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba_Clapper, pos_label='Clapper')
scr_accuracy = round(class_rpt['accuracy'],3)
scr_precision = round(class_rpt['Clapper']['precision'],3)
scr_sensitivity = round(class_rpt['Clapper']['recall'], 3)
scr_specificity = round(class_rpt['King']['recall'], 3)
scr_auc = round(auc(fpr, tpr), 3)
scr_k = round(cohen_kappa_score(y_test, predictions), 3)

results_cols = ['Class', 'Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'AUC', 'K']
lr_results = pd.DataFrame(columns=results_cols)
lr_results.loc['Logistic Regression'] = ['N', scr_accuracy, scr_precision, scr_sensitivity, scr_specificity, scr_auc, scr_k]
display(lr_results)

# Make charts
fig, axes = plt.subplots(2 ,figsize=(5,8))
fig.tight_layout(h_pad=5, w_pad=5)

cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix= cm, display_labels=lr.classes_)
disp.plot(ax=axes[0])
axes[1].plot(fpr,tpr,label="LR AUC="+'{:.3f}'.format(scr_auc))
axes[1].plot([0, 1], [0, 1],linestyle='dashed', color='gray', label="Random classifier")
axes[1].set_xlabel('False Positive Rate (FPR)')
axes[1].set_ylabel('True Positive Rate (TPR)')
plt.legend(loc=4)
'';

In [ ]:
display(pd.concat([ldr_results, qdr_results, lr_results]).sort_values(by='Accuracy', ascending=False))

#### Parametric CV Test

In [ ]:
from sklearn.model_selection import cross_validate, LeaveOneOut, KFold

## LDR
ldr_cv = LinearDiscriminantAnalysis()
precision_scorer = make_scorer(precision_score, pos_label='Clapper')
sensitivity_scorer = make_scorer(recall_score, pos_label='Clapper')
specificity_scorer = make_scorer(recall_score, pos_label='King')
# auc_scorer = make_scorer(roc_auc_score, needs_proba=True)
k_scorer = make_scorer(cohen_kappa_score)
scoring = {'accuracy': 'accuracy',
            'precision': precision_scorer,
            'sensitivity': sensitivity_scorer,
            'specificity': specificity_scorer,
            'auc': 'roc_auc',
            'k': k_scorer}

# loo = LeaveOneOut()
# kfold = KFold(n_splits=10, shuffle=True, random_state=77)
scores = cross_validate(ldr_cv, X, y, scoring=scoring, cv=10)
scr_accuracy = round(scores['test_accuracy'].mean(), 3)
scr_precision = round(scores['test_precision'].mean(), 3)
scr_sensitivity = round(scores['test_sensitivity'].mean(), 3)
scr_specificity = round(scores['test_specificity'].mean(), 3)
scr_auc = round(scores['test_auc'].mean(), 3)
scr_k = round(scores['test_k'].mean(), 3)


results_cols = ['Class', 'Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'AUC', 'K']
ldrcv_results = pd.DataFrame(columns=results_cols)
ldrcv_results.loc['Linear DFA - CV'] = ['N', scr_accuracy, scr_precision, scr_sensitivity, scr_specificity, scr_auc, scr_k]
display(ldrcv_results)

In [ ]:
display(pd.concat([ldr_results, qdr_results, lr_results, ldrcv_results]).sort_values(by='Accuracy', ascending=False))

#### Non-Parametric Models